# Setting up enviorment

In [1]:
!pip install -q transformers datasets accelerate evaluate scikit-learn torch
!pip install -U transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import random
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score


2025-11-10 20:36:23.249380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762806983.428166      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762806983.474367      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import inspect
from transformers import TrainingArguments
print(inspect.signature(TrainingArguments))


(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: float = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str] = <factory>, warmup_ratio: float = 0.0, warmup_steps: int = 0, log

In [4]:
import os, random, numpy as np, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


# Import file and analyze it 

In [5]:
Seed=42
random.seed(Seed)
np.random.seed(Seed)
os.environ["PYTHONHASHSEED"] = str(Seed)


In [6]:
df = pd.read_csv('/kaggle/input/sensitive-analysis/sentiment-analysis.csv', encoding='utf-8-sig')
print("Columns:", df.columns.tolist())
print(df.head(5))

Columns: ['Text', ' Sentiment', ' Source', ' Date/Time', ' User ID', ' Location', ' Confidence Score']
                                               Text  Sentiment         Source  \
0                              I love this product!   Positive        Twitter   
1                         The service was terrible.   Negative   Yelp Reviews   
2                            This movie is amazing!   Positive           IMDb   
3  I'm so disappointed with their customer support.   Negative   Online Forum   
4                Just had the best meal of my life!   Positive    TripAdvisor   

              Date/Time       User ID      Location   Confidence Score  
0   2023-06-15 09:23:14      @user123      New York               0.85  
1   2023-06-15 11:45:32       user456   Los Angeles               0.65  
2   2023-06-15 14:10:22   moviefan789        London               0.92  
3   2023-06-15 17:35:11    forumuser1       Toronto               0.78  
4   2023-06-16 08:50:59      foodie22        

In [7]:
df.columns = df.columns.str.strip()
text_col = 'Text'
label_col = 'Sentiment'

unique = sorted(df[label_col].unique())
label2id = {lab: i for i, lab in enumerate(unique)}
id2label = {i: lab for lab, i in label2id.items()}

df["label"] = df[label_col].map(label2id)

print("Label map:", label2id)
print(df[[text_col, label_col, "label"]].head())


Label map: {' Negative': 0, ' Positive': 1, 'Negative': 2, 'Positive': 3}
                                               Text  Sentiment  label
0                              I love this product!   Positive      1
1                         The service was terrible.   Negative      0
2                            This movie is amazing!   Positive      1
3  I'm so disappointed with their customer support.   Negative      0
4                Just had the best meal of my life!   Positive      1


# Preprocessing
### we may observe our sentiments are positive or negative convert them to digit 0 or 1

# Lets convert our data set into Hugging face format 
### for testing we are using small dataset

In [8]:
data=Dataset.from_pandas(df[[text_col,"label"]])
data_split=data.train_test_split(test_size=0.2,seed=Seed)
dataset=DatasetDict({"train":data_split["train"],"validation":data_split["test"]})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Text', 'label'],
        num_rows: 364
    })
    validation: Dataset({
        features: ['Text', 'label'],
        num_rows: 92
    })
})


# Importing our BERT Model

In [9]:
model_base = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_base)

def preprocess(batch):
    return tokenizer(batch[text_col], padding="max_length", max_length=128, truncation=True)

dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/364 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

In [10]:
num_labels=len(label2id)
model=AutoModelForSequenceClassification.from_pretrained(model_base,num_labels=num_labels,id2label=id2label,label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# evaulation metrices
from inspect import signature

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

In [12]:
# hugging face default parameters for model training 
training_args = TrainingArguments(
    output_dir="./bert_sentiment",
    learning_rate=2e-5,
    eval_strategy="epoch", 
    save_strategy="epoch",
    
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_39/1921591762.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
import transformers
print(transformers.__version__)


4.57.1


In [14]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.733700,0.478617,1.000000,1.000000
2,0.107400,0.059830,1.000000,1.000000
3,0.024000,0.013668,1.000000,1.000000
4,0.009700,0.006696,1.000000,1.000000
5,0.006500,0.004742,1.000000,1.000000
6,0.004900,0.003764,1.000000,1.000000
7,0.004100,0.003216,1.000000,1.000000
8,0.004400,0.002884,1.000000,1.000000
9,0.003800,0.002724,1.000000,1.000000
10,0.003000,0.002656,1.000000,1.000000


TrainOutput(global_step=460, training_loss=0.1289006680250168, metrics={'train_runtime': 91.6276, 'train_samples_per_second': 39.726, 'train_steps_per_second': 5.02, 'total_flos': 239435359887360.0, 'train_loss': 0.1289006680250168, 'epoch': 10.0})

In [15]:
pred_out = trainer.predict(dataset["validation"])
preds = np.argmax(pred_out.predictions, axis=-1)
labels = pred_out.label_ids

print("Accuracy:", accuracy_score(labels, preds))
print("Macro F1:", f1_score(labels, preds, average="macro"))
label_names = [id2label[i] for i in sorted(set(labels))] 
print("\nClassification Report:\n", classification_report(labels, preds, target_names=label_names))
print("\nConfusion Matrix:\n", confusion_matrix(labels, preds))


Accuracy: 1.0
Macro F1: 1.0

Classification Report:
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00         4
    Positive       1.00      1.00      1.00        13
    Negative       1.00      1.00      1.00        35
    Positive       1.00      1.00      1.00        40

    accuracy                           1.00        92
   macro avg       1.00      1.00      1.00        92
weighted avg       1.00      1.00      1.00        92


Confusion Matrix:
 [[ 4  0  0  0]
 [ 0 13  0  0]
 [ 0  0 35  0]
 [ 0  0  0 40]]


In [16]:
trainer.save_model("./bert_sentiment/final")
tokenizer.save_pretrained("./bert_sentiment/final")
print("Saved to ./bert_sentiment/final")


Saved to ./bert_sentiment/final


In [17]:
import torch

test_texts = [
    "I thought it would be terrible but it exceeded my expectations!",
    "The product isn't bad but the customer service ruined everything.",
    "Not the best, not the worst, just okay I guess.",
    "I wanted to love it so badly but it let me down completely.",
    "Despite the flaws, I'm surprisingly happy with my purchase.",
    "The reviews said it was amazing but I found it utterly disappointing.",
    "It's fine if you have low expectations, otherwise you'll hate it.",
    "I can't believe how much I regret buying this piece of junk.",
    "Mixed feelings - great price but questionable quality.",
    "Honestly expected nothing and still managed to be impressed!",
    "Would have been perfect if not for that one major issue.",
    "I'm not sure if I like it or not, very confusing experience.",
    "Started great but ended up being a waste of money.",
    "The worst part? It actually works but I still hate using it.",
    "Surprisingly decent for the price, can't complain much."
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_inputs = tokenizer(test_texts, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
test_inputs = {k: v.to(device) for k, v in test_inputs.items()}

model.eval()
with torch.no_grad():
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

results = pd.DataFrame({
    "Text": test_texts,
    "Predicted_Label": [id2label[pred.item()] for pred in predictions],
    "Confidence": torch.softmax(outputs.logits, dim=-1).max(dim=-1).values.cpu().numpy()
})

print(results.to_string(index=False))

                                                                 Text Predicted_Label  Confidence
      I thought it would be terrible but it exceeded my expectations!        Negative    0.367108
    The product isn't bad but the customer service ruined everything.        Negative    0.393033
                      Not the best, not the worst, just okay I guess.        Negative    0.320945
          I wanted to love it so badly but it let me down completely.        Negative    0.352130
          Despite the flaws, I'm surprisingly happy with my purchase.        Negative    0.348664
The reviews said it was amazing but I found it utterly disappointing.        Negative    0.385208
    It's fine if you have low expectations, otherwise you'll hate it.        Negative    0.348027
         I can't believe how much I regret buying this piece of junk.        Negative    0.343019
               Mixed feelings - great price but questionable quality.        Negative    0.604844
         Honestly ex

In [18]:
from google.colab import files
import shutil
shutil.make_archive('bert_sentiment_model', 'zip', './bert_sentiment/final')

'/kaggle/working/bert_sentiment_model.zip'